# Query Reward Unit Test
Performing verification of the query rewards mechanism by comparing the event data to the results of cadCAD model.

In [ ]:
import pandas as pd
import numpy as np
from decimal import Decimal
from decimal import *
import model.parts.utils as utils
from model.sys_params import *
df = pd.read_pickle(r'experiment.p')

Configure environment

In [ ]:
df.reset_index(inplace = True)
pd.set_option('display.max_rows', None)

In [ ]:
# print(query_fee_events)
query_fee_events_dict = allocation_collected_events
print("EXPECTED TRUE QUERY REWARDS FROM STAKE DEPOSITED EVENTS:")
for timestep, query_fee_events_list in query_fee_events_dict.items():
    if query_fee_events_list is None:
        query_fee_amt = 0
    else:
        query_fee_amt = sum([e['tokens'] for e in query_fee_events_list])
    print(f"{timestep}, {query_fee_amt}")
    


In [ ]:
print("MODELED RESULTS")
for timestep, query_fee_events_list in query_fee_events_dict.items():    
    event = query_fee_events_list[0]
    new_query_fee_amt = df.iloc[timestep-1].indexers[event['indexer']].cumulative_query_revenue
    old_query_fee_amt = df.iloc[timestep-2].indexers[event['indexer']].cumulative_query_revenue
    query_fee_amt = new_query_fee_amt - old_query_fee_amt
    print(f"{timestep}, {query_fee_amt=}")


In [ ]:
print("UNITTEST RESULTS")
cntExact = 0
cntReallyClose = 0
cntClose = 0
cnt = 0
cntWrong = 0
for timestep, query_fee_events_list in query_fee_events_dict.items():
    if query_fee_events_list is None:
        event_query_fee_amt = 0
    else:
        event_query_fee_amt = sum([e['tokens'] for e in query_fee_events_list])
    event = query_fee_events_list[0]
    new_query_fee_amt = df.iloc[timestep-1].indexers[event['indexer']].cumulative_query_revenue
    old_query_fee_amt = df.iloc[timestep-2].indexers[event['indexer']].cumulative_query_revenue
    model_query_fee_amt = new_query_fee_amt - old_query_fee_amt
    print(f"Event: Timestep={timestep-1}, Query Reward Tokens={event_query_fee_amt}")
    print(f"Model: Timestep={timestep-1}, Query Reward Tokens={model_query_fee_amt}")
    # print("Query Rewards Equal?", event_query_fee_amt == model_query_fee_amt)
    # print()

    try:
        ratio =  abs(event_query_fee_amt / model_query_fee_amt)
    except:
        ratio = np.inf
    exact = ratio == 1.0
    close = 0.99 <= ratio <= 1.01
    reallyClose = 0.99999 <= ratio <= 1.00001

    if exact:            
        cntExact += 1
    elif reallyClose:
        cntReallyClose += 1
    elif close:
        cntClose += 1
    else:
        cntWrong += 1
    cnt += 1
    print("Tokens Equal?", exact)
    print("Tokens Within 0.001%?", reallyClose)
    print("Tokens Within 1%?", close)
    print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    print()
print(f"Exact: {cntExact}, ReallyClose: {cntReallyClose}, Close: {cntClose}, Wrong: {cntWrong}, Total Number: {cnt}")   
    
    


# Conclusion
The query reward token quantities are exactly equal.